<h1>Creating and preparing a bank marketing database</h1>

<p>
from a csv file.
</p>

<p>
This project is based on a
<a href ="https://www.datacamp.com">
Datacamp project.
</a>
</p>

<center><img src="https://assets.datacamp.com/production/repositories/6005/datasets/0c64652120b80cfe2762012d7252b439138be223/piggy_bank.jpg" alt="Piggy bank" width="600" height="500"></center>

Personal loans are a lucrative revenue stream for banks. The typical interest rate of a two year loan in the United Kingdom is [around 10%](https://www.experian.com/blogs/ask-experian/whats-a-good-interest-rate-for-a-personal-loan/). This might not sound like a lot, but in September 2022 alone UK consumers borrowed [around £1.5 billion](https://www.ukfinance.org.uk/system/files/2022-12/Household%20Finance%20Review%202022%20Q3-%20Final.pdf)!

You have been asked to work with a bank to clean and store the data they collected as part of a recent marketing campaign, which aimed to get customers to take out a personal loan. They plan to conduct more marketing campaigns going forward so would like you to set up a PostgreSQL database to store this campaign's data, designing the schema in a way that would allow data from future campaigns to be easily imported. 

They have supplied you with a csv file called `"bank_marketing.csv"`, which you will need to clean, reformat, and split, in order to save separate files based on the tables you will create. Lastly, you will write the SQL code that the bank can execute to create the tables and populate with the data from the csv files. As the bank are quite strict about their security, you'll provide the database design script as a `.sql` file that they can then run. 

You have been asked to design a database that will have three tables:

## `client`
| column | data type | description |
|--------|-----------|-------------|
| `id` | `serial` | Client ID - primary key |
| `age` | `integer` | Client's age in years |
| `job` | `text` | Client's type of job |
| `marital` | `text` | Client's marital status |
| `education` | `text` | Client's level of education |
| `credit_default` | `text` | Whether the client's credit is in default |
| `housing` | `text` | Whether the client has an existing housing loan (mortgage) |
| `loan` | `text` | Whether the client has an existing personal loan |

## `campaign`
| column | data type | description |
|--------|-----------|-------------|
| `campaign_id` | `integer` | Campaign ID |
| `client_id` | `serial` | Client ID - references `id` in the `client` table |
| `number_contacts` | `integer` | Number of contact attempts to the client in the current campaign |
| `contact_duration` | `integer` | Last contact duration in seconds |
| `pdays` | `integer` | Number of days since contact in previous campaign (`999` = not previously contacted) |
| `previous_campaign_contacts` | `integer` | Number of contact attempts to the client in the previous campaign |
| `previous_outcome` | `text` | Outcome of the previous campaign |
| `campaign_outcome` | `integer` | Outcome of the current campaign |
| `last_contact_date` | `date` | Last date the client was contacted |

## `economics`
| column | data type | description |
|--------|-----------|-------------|
| `client_id` | `serial` | Client ID - references `id` in the `client` table |
| `emp_var_rate` | `float` | Employment variation rate (quarterly indicator) |
| `cons_price_idx` | `float` | Consumer price index (monthly indicator) |
| `euribor_three_months` | `float` | Euro Interbank Offered Rate (euribor) three month rate (daily indicator) |
| `number_employed` | `float` | Number of employees (quarterly indicator)| 

## Import packages

In [ ]:
# Start coding...
import pandas as pd
import numpy as np

## Import data

In [40]:
bank=pd.read_csv("bank_marketing.csv")
print(bank.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 22 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   client_id       41188 non-null  int64  
 1   age             41188 non-null  int64  
 2   job             41188 non-null  object 
 3   marital         41188 non-null  object 
 4   education       41188 non-null  object 
 5   credit_default  41188 non-null  object 
 6   housing         41188 non-null  object 
 7   loan            41188 non-null  object 
 8   contact         41188 non-null  object 
 9   month           41188 non-null  object 
 10  day_of_week     41188 non-null  object 
 11  duration        41188 non-null  int64  
 12  campaign        41188 non-null  int64  
 13  pdays           41188 non-null  int64  
 14  previous        41188 non-null  int64  
 15  poutcome        41188 non-null  object 
 16  emp_var_rate    41188 non-null  float64
 17  cons_price_idx  41188 non-null 

## Data wrangling & cleaning

In [41]:
bank['year'] =2022
bank['year'].astype(int)

0        2022
1        2022
2        2022
3        2022
4        2022
         ... 
41183    2022
41184    2022
41185    2022
41186    2022
41187    2022
Name: year, Length: 41188, dtype: int32

In [42]:
bank.head()

,client_id,age,job,marital,education,credit_default,housing,loan,contact,month,...,pdays,previous,poutcome,emp_var_rate,cons_price_idx,cons_conf_idx,euribor3m,nr_employed,y,year
0,0,56,housemaid,married,basic.4y,no,no,no,telephone,may,...,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no,2022
1,1,57,services,married,high.school,unknown,no,no,telephone,may,...,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no,2022
2,2,37,services,married,high.school,no,yes,no,telephone,may,...,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no,2022
3,3,40,admin.,married,basic.6y,no,no,no,telephone,may,...,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no,2022
4,4,56,services,married,high.school,no,no,yes,telephone,may,...,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no,2022


In [43]:
bank.dtypes

client_id           int64
age                 int64
job                object
marital            object
education          object
credit_default     object
housing            object
loan               object
contact            object
month              object
day_of_week        object
duration            int64
campaign            int64
pdays               int64
previous            int64
poutcome           object
emp_var_rate      float64
cons_price_idx    float64
cons_conf_idx     float64
euribor3m         float64
nr_employed       float64
y                  object
year                int64
dtype: object

In [44]:
bank.isnull().sum()

client_id         0
age               0
job               0
marital           0
education         0
credit_default    0
housing           0
loan              0
contact           0
month             0
day_of_week       0
duration          0
campaign          0
pdays             0
previous          0
poutcome          0
emp_var_rate      0
cons_price_idx    0
cons_conf_idx     0
euribor3m         0
nr_employed       0
y                 0
year              0
dtype: int64

In [45]:
bank["education"] = bank['education'].str.replace(".", "_", regex=True)

bank["education"].fillna(value={'unknown':np.NaN}, inplace=True)

if 'unknown' in bank["education"]:
    print("Found unknown")
else:
    print("Unknown not found.")

Unknown not found.


In [46]:
bank["education"].isnull().sum()

0

In [47]:
"." in  bank["education"]

False

In [48]:
bank['job']=bank['job'].str.replace('.', '')

print(type(bank['job']))
print(bank['job'].head())

if "." in bank['job']:
    print("Period found in job variable.")
else:
    print("Period not found in job variable.")

<class 'pandas.core.series.Series'>
0    housemaid
1     services
2     services
3        admin
4     services
Name: job, dtype: object
Period not found in job variable.


C:\Users\rolfc\AppData\Local\Temp/ipykernel_7376/1326380922.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  bank['job']=bank['job'].str.replace('.', '')


In [49]:
bank['poutcome'].value_counts()

nonexistent    35563
failure         4252
success         1373
Name: poutcome, dtype: int64

In [50]:
bank['poutcome']=\
bank['poutcome'].map({'success':1, 'failure':0, 'nonexistent':np.NaN})

bank['poutcome'].value_counts()

0.0    4252
1.0    1373
Name: poutcome, dtype: int64

In [51]:
bank['y'].value_counts()

no     36548
yes     4640
Name: y, dtype: int64

In [52]:
bank['y'] = bank['y'].map({'no':0, 'yes':1})
bank['y'].value_counts()

0    36548
1     4640
Name: y, dtype: int64

In [53]:
economics =  \
bank.loc[:,["client_id" , "emp_var_rate", "cons_price_idx", "euribor3m", "nr_employed"]]

print(economics.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   client_id       41188 non-null  int64  
 1   emp_var_rate    41188 non-null  float64
 2   cons_price_idx  41188 non-null  float64
 3   euribor3m       41188 non-null  float64
 4   nr_employed     41188 non-null  float64
dtypes: float64(4), int64(1)
memory usage: 1.6 MB
None


In [54]:
  
economics.rename(columns={"euribor3m":"euribor_three_months", 
                          "nr_employed":"number_employed"}, 
                 inplace=True)

In [55]:
economics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 5 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   client_id             41188 non-null  int64  
 1   emp_var_rate          41188 non-null  float64
 2   cons_price_idx        41188 non-null  float64
 3   euribor_three_months  41188 non-null  float64
 4   number_employed       41188 non-null  float64
dtypes: float64(4), int64(1)
memory usage: 1.6 MB


### Renaming column names

In [56]:

client = bank.loc[:, ["client_id",
"age",
"job",
"marital",
"education",
"credit_default",	
"housing",
"loan"]]

client.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   client_id       41188 non-null  int64 
 1   age             41188 non-null  int64 
 2   job             41188 non-null  object
 3   marital         41188 non-null  object
 4   education       41188 non-null  object
 5   credit_default  41188 non-null  object
 6   housing         41188 non-null  object
 7   loan            41188 non-null  object
dtypes: int64(2), object(6)
memory usage: 2.5+ MB


In [57]:
client.rename(columns={'client_id':'id'}, inplace=True)

client.head()

,id,age,job,marital,education,credit_default,housing,loan
0,0,56,housemaid,married,basic_4y,no,no,no
1,1,57,services,married,high_school,unknown,no,no
2,2,37,services,married,high_school,no,yes,no
3,3,40,admin,married,basic_6y,no,no,no
4,4,56,services,married,high_school,no,no,yes


In [58]:
campaign = bank.loc[:, ["client_id", "campaign", "duration", 
                        "pdays","previous","poutcome", "y"]]

campaign.insert(0, "campaign_id", 1)


print(campaign.info())
print(campaign.head())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   campaign_id  41188 non-null  int64  
 1   client_id    41188 non-null  int64  
 2   campaign     41188 non-null  int64  
 3   duration     41188 non-null  int64  
 4   pdays        41188 non-null  int64  
 5   previous     41188 non-null  int64  
 6   poutcome     5625 non-null   float64
 7   y            41188 non-null  int64  
dtypes: float64(1), int64(7)
memory usage: 2.5 MB
None
   campaign_id  client_id  campaign  duration  pdays  previous  poutcome  y
0            1          0         1       261    999         0       NaN  0
1            1          1         1       149    999         0       NaN  0
2            1          2         1       226    999         0       NaN  0
3            1          3         1       151    999         0       NaN  0
4            1          4         1  

In [59]:
campaign.rename(columns={"duration":"contact_duration", "previous": "previous_campaign_contacts", "poutcome": "previous_outcome", "campaign":"number_contacts", "y":"campaign_outcome",}, inplace=True)

In [60]:
campaign=campaign[["client_id",   
"contact_duration",    
"number_contacts",    
"pdays",       
"previous_campaign_contacts",       
"previous_outcome" ,
"campaign_outcome"]]

In [61]:
campaign.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 7 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   client_id                   41188 non-null  int64  
 1   contact_duration            41188 non-null  int64  
 2   number_contacts             41188 non-null  int64  
 3   pdays                       41188 non-null  int64  
 4   previous_campaign_contacts  41188 non-null  int64  
 5   previous_outcome            5625 non-null   float64
 6   campaign_outcome            41188 non-null  int64  
dtypes: float64(1), int64(6)
memory usage: 2.2 MB


In [62]:
bank['month'].value_counts()

may    13769
jul     7174
aug     6178
jun     5318
nov     4101
apr     2632
oct      718
sep      570
mar      546
dec      182
Name: month, dtype: int64

## Create a datetime variable

In [63]:
bank.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 23 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   client_id       41188 non-null  int64  
 1   age             41188 non-null  int64  
 2   job             41188 non-null  object 
 3   marital         41188 non-null  object 
 4   education       41188 non-null  object 
 5   credit_default  41188 non-null  object 
 6   housing         41188 non-null  object 
 7   loan            41188 non-null  object 
 8   contact         41188 non-null  object 
 9   month           41188 non-null  object 
 10  day_of_week     41188 non-null  object 
 11  duration        41188 non-null  int64  
 12  campaign        41188 non-null  int64  
 13  pdays           41188 non-null  int64  
 14  previous        41188 non-null  int64  
 15  poutcome        5625 non-null   float64
 16  emp_var_rate    41188 non-null  float64
 17  cons_price_idx  41188 non-null 

In [64]:
bank['month']=bank['month'].str.capitalize()
bank['day_of_week']=bank['day_of_week'].str.capitalize()


In [65]:

last_contact_date_1 = "2022" + "-" + bank['month'] + "-" + bank['day_of_week']

last_contact_date_1.head()

0    2022-May-Mon
1    2022-May-Mon
2    2022-May-Mon
3    2022-May-Mon
4    2022-May-Mon
dtype: object

In [66]:
last_contact_date_datetime = \
pd.to_datetime(last_contact_date_1, format="%Y-%b-%a")

bank["last_contact_date"] = last_contact_date_datetime 

bank.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 24 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   client_id          41188 non-null  int64         
 1   age                41188 non-null  int64         
 2   job                41188 non-null  object        
 3   marital            41188 non-null  object        
 4   education          41188 non-null  object        
 5   credit_default     41188 non-null  object        
 6   housing            41188 non-null  object        
 7   loan               41188 non-null  object        
 8   contact            41188 non-null  object        
 9   month              41188 non-null  object        
 10  day_of_week        41188 non-null  object        
 11  duration           41188 non-null  int64         
 12  campaign           41188 non-null  int64         
 13  pdays              41188 non-null  int64         
 14  previo

In [67]:
bank.head()

,client_id,age,job,marital,education,credit_default,housing,loan,contact,month,...,previous,poutcome,emp_var_rate,cons_price_idx,cons_conf_idx,euribor3m,nr_employed,y,year,last_contact_date
0,0,56,housemaid,married,basic_4y,no,no,no,telephone,May,...,0,NaN,1.1,93.994,-36.4,4.857,5191.0,0,2022,2022-05-01
1,1,57,services,married,high_school,unknown,no,no,telephone,May,...,0,NaN,1.1,93.994,-36.4,4.857,5191.0,0,2022,2022-05-01
2,2,37,services,married,high_school,no,yes,no,telephone,May,...,0,NaN,1.1,93.994,-36.4,4.857,5191.0,0,2022,2022-05-01
3,3,40,admin,married,basic_6y,no,no,no,telephone,May,...,0,NaN,1.1,93.994,-36.4,4.857,5191.0,0,2022,2022-05-01
4,4,56,services,married,high_school,no,no,yes,telephone,May,...,0,NaN,1.1,93.994,-36.4,4.857,5191.0,0,2022,2022-05-01


In [68]:
bank['last_contact_date'].value_counts()

2022-05-01    13769
2022-07-01     7174
2022-08-01     6178
2022-06-01     5318
2022-11-01     4101
2022-04-01     2632
2022-10-01      718
2022-09-01      570
2022-03-01      546
2022-12-01      182
Name: last_contact_date, dtype: int64

In [69]:
campaign["last_contact_date"] = last_contact_date_datetime 

campaign.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 8 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   client_id                   41188 non-null  int64         
 1   contact_duration            41188 non-null  int64         
 2   number_contacts             41188 non-null  int64         
 3   pdays                       41188 non-null  int64         
 4   previous_campaign_contacts  41188 non-null  int64         
 5   previous_outcome            5625 non-null   float64       
 6   campaign_outcome            41188 non-null  int64         
 7   last_contact_date           41188 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(6)
memory usage: 2.5 MB


In [70]:
campaign.head()

,client_id,contact_duration,number_contacts,pdays,previous_campaign_contacts,previous_outcome,campaign_outcome,last_contact_date
0,0,261,1,999,0,NaN,0,2022-05-01
1,1,149,1,999,0,NaN,0,2022-05-01
2,2,226,1,999,0,NaN,0,2022-05-01
3,3,151,1,999,0,NaN,0,2022-05-01
4,4,307,1,999,0,NaN,0,2022-05-01


## Store csv files

In [71]:
campaign.to_csv('campaign.csv', index=False)
economics.to_csv('economics.csv', index=False)
client.to_csv('client.csv', index=False)

In [72]:
campaign.head(2)

,client_id,contact_duration,number_contacts,pdays,previous_campaign_contacts,previous_outcome,campaign_outcome,last_contact_date
0,0,261,1,999,0,NaN,0,2022-05-01
1,1,149,1,999,0,NaN,0,2022-05-01


In [73]:
economics.head(2)

,client_id,emp_var_rate,cons_price_idx,euribor_three_months,number_employed
0,0,1.1,93.994,4.857,5191.0
1,1,1.1,93.994,4.857,5191.0


In [74]:
client.head(2)

,id,age,job,marital,education,credit_default,housing,loan
0,0,56,housemaid,married,basic_4y,no,no,no
1,1,57,services,married,high_school,unknown,no,no


## Create database design
consisting of 3 tables.

In [75]:
# Store and print database_design
database_design = \
{'client': client, 'campaign':campaign, 'economics':economics }


print(database_design)

{'client':           id  age          job  marital            education credit_default  \
0          0   56    housemaid  married             basic_4y             no   
1          1   57     services  married          high_school        unknown   
2          2   37     services  married          high_school             no   
3          3   40        admin  married             basic_6y             no   
4          4   56     services  married          high_school             no   
...      ...  ...          ...      ...                  ...            ...   
41183  41183   73      retired  married  professional_course             no   
41184  41184   46  blue-collar  married  professional_course             no   
41185  41185   56      retired  married    university_degree             no   
41186  41186   44   technician  married  professional_course             no   
41187  41187   74      retired  married  professional_course             no   

      housing loan  
0          no   no 